<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-задачи" data-toc-modified-id="Описание-задачи-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание задачи</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Описание задачи

В данном задании необходимо разработать модель которая позволит выявлять токсичные комментарии комментарии на сайте. Критерием достижения результата является значение метрики F1 не менее 0.75
Последовательность выполнения задач будет стандартной:
1. Загрузка и подготовка данных
2. Обучение моделей
3. Сравнение результатов и формулирование выводов

## Подготовка данных

Загрузим библиотеки

In [141]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Загрузим данные

In [142]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [143]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Колонка Unnamed: 0 является неинформативной. Ее можно удалить

In [144]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [145]:
data.describe()

,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


В файле порядка 160 тысяч комментариев, 10% из которых являются токсичными. Проведем очистку и лемматизацию предварительно создав соответствующие функции

In [146]:
def clear_text(text):
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = ' '.join(clear_text.split())
    return clear_text

In [147]:
data['text'] = data['text'].apply(clear_text)
data.head()

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He matches this background colour I m se...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0


In [148]:
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

In [149]:
data['lemm_text'] = data['text'].apply(lemmatize_words)
data.head()

,text,toxic,lemm_text
0,Explanation Why the edits made under my userna...,0,Explanation Why the edit make under my usernam...
1,D aww He matches this background colour I m se...,0,D aww He match this background colour I m seem...
2,Hey man I m really not trying to edit war It s...,0,Hey man I m really not try to edit war It s ju...
3,More I can t make any real suggestions on impr...,0,More I can t make any real suggestions on impr...
4,You sir are my hero Any chance you remember wh...,0,You sir be my hero Any chance you remember wha...


Разобьем выборку на обучающую и тестовую в стандартном соотношении 80:20

In [150]:
data.describe()

,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [151]:
data_train, data_test = train_test_split(data, test_size=0.2)
data_train.shape

(127433, 3)

In [152]:
data_train, data_valid = train_test_split(data_train, test_size=0.25)
data_valid.shape

(31859, 3)

In [153]:
data_train.shape

(95574, 3)

Проведем очистку данных от стоп-слов и их векторизацию

In [154]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [155]:
tf_idf_train = count_tf_idf.fit_transform(data_train['lemm_text'])
tf_idf_valid = count_tf_idf.transform(data_valid['lemm_text'])
tf_idf_test = count_tf_idf.transform(data_test['lemm_text'])

In [156]:
tf_idf_train.shape

(95574, 119372)

In [157]:
features_train = tf_idf_train
target_train = data_train['toxic']
features_valid = tf_idf_valid
target_valid = data_valid['toxic']
features_test = tf_idf_test
target_test = data_test['toxic']

## Обучение

По условиям задачи необходимо попробовать несколько моделей. Применим логистическую регрессию и случайный лес с перебором количества деревьев и максимальной глубины

### Логистическая регрессия

In [158]:
model = LogisticRegression(random_state=12345, C=5, max_iter=1000)
model.fit(features_train, target_train)
predictions = model.predict(features_train)
f1 = f1_score(predictions, target_train)
print("Значение метрики f1", f1)

Значение метрики f1 0.8700641241984475


Полученное на обучающей выборке значение f1 превышает целевое. Попробуем вторую модель - случайный лес

### Случайный лес

In [159]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 10):
    for depth in range (1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = f1_score(predictions, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print("Значение метрики f1", best_result, 'при количестве деревьев', best_est, 'и максимальной глубине', best_depth)          

Значение метрики f1 0.06612529002320185 при количестве деревьев 1 и максимальной глубине 9


Полученное значение  f1 в 0.06 значительно ниже целевого. Модель может быть отброшена

Проверим лучшее из получившихся решений, логистическую регрессию на тестовой выборке

In [160]:
model = LogisticRegression(random_state=12345, C=5, max_iter=1000)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
f1 = f1_score(predictions, target_test)
print("Значение метрики f1", f1)

Значение метрики f1 0.7662361287975259


Полученное значение метрики f1 находится выше минимального значения в 0.75. Модель может быть рекомендована к использованию

## Выводы

В рамках поставленной задачи по выбору оптимальной модели для сортировки комментариев было проведено сравнение двух моделей: логистической регрессии и дерева решений с подбором гиперпараметров. Дерево решений при оптимальном сочетании гиперпараметров позволяет достичь показателя f1 в 0.06 что существенно ниже целевого показателя. У логистической регрессии показатель f1 на тестовой выборке составляет 0.76 Это позволяет рекомендовать ее для использования в данной задаче